In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from tqdm import tqdm,tqdm_notebook

from sklearn import preprocessing
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.linear_model import LinearRegression

## Read CSV's

In [2]:
movies = pd.read_csv('comp-data/4a-training-dataset-creation/movies.csv')
users = pd.read_csv('comp-data/4a-training-dataset-creation/users.csv')
ratings3 = pd.read_csv('comp-data/4a-training-dataset-creation/ratings3.csv')
ratings4 = pd.read_csv('comp-data/4a-training-dataset-creation/ratings4.csv')

## Create the two Aprroaches of the Dataset
  * The first one implies that people who rated a movie with **2 or less** stars are **most likely not going to see that movie in the first place**
  * The second one implies that people who rated a movie with **3 or less** stars are **most likely not going to see that movie in the first place**

In [3]:
df3 = ((users.merge(ratings3, how='inner', on='user id'))\
.merge(movies, how='inner', left_on='item id', right_on='movie id'))\
.drop(['item id', 'movie id', 'user id'], axis=1)

In [8]:
df3.head(10)

,age,gender,zip code,ocupation,rating,unknown,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,release-year
0,2,0,85711,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1994
1,4,0,29206,7,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1994
2,3,1,37212,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1994
3,2,0,52246,10,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1994
4,4,0,08403,7,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1994
5,5,0,06472,17,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1994
6,2,0,02215,5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1994
7,2,0,71457,5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1994
8,5,0,20910,10,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1994
9,2,0,53703,5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1994


In [4]:
df4 = ((users.merge(ratings4, how='inner', on='user id'))\
.merge(movies, how='inner', left_on='item id', right_on='movie id'))\
.drop(['item id', 'movie id', 'user id'], axis=1)

In [20]:
df4.shape

(99991, 25)

## Normalize the Dataset so that we eliminate outliers
  * Final Features: {age, gender, zip code, occupation, target, movie_genres (one-hot representation), release year}

In [5]:
# Normalize the Dataset
X = np.array(df3.drop(['rating', 'zip code'], axis=1))
X = preprocessing.scale(X)
# Separate the correct answer y from the Dataset
y = np.array(df3['rating'])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


## Initiate the Repeated K-fold Validation Learning Technique
  * for K = 5 (a.k.a 5 shards of the learning dataset)
  * and for 50 repeats (so as to have concrete proof of the machine's accuracy

In [6]:
#kf = KFold(n_splits=5)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
rkf = RepeatedKFold(n_splits=5, n_repeats=50, random_state=100)

## Initiate the Least Squares Algorithm

In [7]:
clf = LinearRegression()
accuracy = np.array([])

for train_index, test_index in tqdm_notebook(rkf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    accuracy = np.append(accuracy, [clf.score(X_test,y_test)])

print(np.amax(accuracy))
print(np.amin(accuracy))

A Jupyter Widget


0.0381592075707
-3.57910076257


## Beta Notes

In [ ]:
tmp = df3.copy()
one_hot = pd.get_dummies(tmp['age'], prefix='age')
tmp = tmp.drop(['age'], axis=1)
tmp = tmp.join(one_hot)
tmp.head(10)

In [ ]:
one_hot = pd.get_dummies(tmp['ocupation'], prefix='occ')
tmp = tmp.drop(['ocupation'], axis=1)
tmp = tmp.join(one_hot)
tmp.head(10)